In [ ]:
%env PYTHONHASHSEED=0

In [ ]:
import numpy as np
import pandas as pd
import os

import scanpy as sc
from anndata import AnnData, concat
from matplotlib.transforms import Bbox

# from scvi.model.utils import mde

import scvi 
from matplotlib import pyplot as plt
import seaborn as sns

from scipy import sparse
import gc
# from upsetplot import plot as UpSet
from itertools import product
import networkx as nx

from scipy.cluster import hierarchy

import re
import subprocess

import h5py

import itertools

# from scnym.api import scnym_api

# import pymn

from collections import Counter

In [ ]:
from scvi.model.utils import mde


In [ ]:
out_dir = "/home/anna/ClusterProjects/SA_bone_marrow/scfiles/"

In [ ]:
# import tensorflow as tf
# import tensorboard as tb
# tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

pd.set_option('display.max_columns', 500)

In [ ]:
### Directory TM_combined_datasets_SA_ann_obs
# combined_datasets = sc.read_h5ad(filename = "../data/h5ad/TM_combined_datasets_SA_ann.h5ad")
# obs_plk = pd.read_pickle("../data/h5ad/TM_combined_datasets_SA_ann_obs.plk")
# combined_datasets.obs = obs_plk

In [ ]:
hvg = sc.read("../data/h5ad/hvg_integrated_170824.h5ad")

In [ ]:
hvg.obsm

### Extracting only SA experiment from the scNym datasets

In [ ]:
# SA_exps = combined_datasets[combined_datasets.obs["study_id"].isin(['SA_pilot', 'SA_final'])]

### Reading SA experiment

In [ ]:
SA_exps = sc.read_h5ad(filename = out_dir + "SA_exp_scNym_annotated.h5ad")
obs_plk = pd.read_pickle(out_dir + "SA_exp_scNym_annotated.plk")
SA_exps.obs = obs_plk

In [ ]:
hvg_cells = hvg.obs_names
SA_exp_names = SA_exps.obs_names

SA_hvg_cells = [x for x in SA_exp_names if x[:-2] in hvg_cells]

SA_cellname_counts = Counter([x[:-2] for x in SA_hvg_cells])
SA_unique_names = [x for x in SA_hvg_cells if SA_cellname_counts[x[:-2]] == 1]

SA_exps_subset = SA_exps[SA_unique_names]

SA_exps_subset.obs_names = [x[:-2] for x in SA_exps_subset.obs_names]

hvg_subset = hvg[SA_exps_subset.obs_names]

hvg_subset.obs["scNym_condensed"] = SA_exps_subset.obs["scNym_condensed"]
hvg_subset.obs["scNym_condensed_confidence"] = SA_exps_subset.obs["scNym_condensed_confidence"]

In [ ]:
SA_exps_HSC = SA_exps_subset[SA_exps_subset.obs["scNym_condensed"] == "hematopoietic precursor cell"]
SA_exps_mature = SA_exps_subset[SA_exps_subset.obs["scNym_condensed"] != "hematopoietic precursor cell"]

### HSC population

In [ ]:
hvg_HSC_v2 = hvg_subset[SA_exps_HSC.obs_names.intersection(hvg_subset.obs_names)]
# SA_HSC_v2 = SA_exps_HSC[SA_exps_HSC.obs_names.intersection(hvg_subset.obs_names)]

In [ ]:
hvg_HSC_v2.shape

In [ ]:
hvg_HSC_v2.layers

### Loading the TM HSC SA annotated dataset

In [ ]:
combined_HSC = sc.read_loom(filename = "../data/loom/TM_combined_HSC_SA_ann.loom")

In [ ]:
annotated_SA_HSC = combined_HSC[ combined_HSC.obs["study_id"].isin(["SA_final", "SA_pilot"])]
annotated_SA_HSC.obs_names = annotated_SA_HSC.obs["obs_names"]

In [ ]:
del(combined_HSC)

In [ ]:
ann_counts = Counter([x[:-4] for x in annotated_SA_HSC.obs_names])
unique_names = [x for x in annotated_SA_HSC.obs_names if ann_counts[x[:-4]] == 1]
annotated_SA_HSC = annotated_SA_HSC[unique_names]
annotated_SA_HSC.obs_names = [x[:-4] for x in annotated_SA_HSC.obs_names]
common_cells = annotated_SA_HSC.obs_names.intersection(hvg_HSC_v2.obs_names)

In [ ]:
Counter(hvg_HSC_v2.obs["scNym_HPC"])

In [ ]:
hvg_HSC_v2.obs["scNym_HPC"] = annotated_SA_HSC[common_cells].obs["scNym_HPC"]

In [ ]:
sum(common_cells == hvg_HSC_v2.obs_names)

In [ ]:
# sc.pp.normalize_total(hvg_HSC_v2, target_sum=1e6)
# sc.pp.log1p(hvg_HSC_v2)

In [ ]:
hvg_HSC_v2.shape

In [ ]:
hvg_HSC_v2.layers

In [ ]:
hvg_HSC_v2.obsm["X_scVI"]

In [ ]:
hvg_HSC_v2.layers

In [ ]:
sc.tl.pca(hvg_HSC_v2, svd_solver='arpack', layer = "scVI_normalized")
sc.pp.neighbors(hvg_HSC_v2, n_neighbors=10, n_pcs=40, use_rep = "X_pca")
sc.tl.umap(hvg_HSC_v2, random_state=4567898)
sc.tl.leiden(hvg_HSC_v2, resolution=0.25, 
             n_iterations=100)

In [ ]:
hvg_HSC_v2.obs

In [ ]:
sc.set_figure_params(figsize = (8,6))
sc.pl.embedding(
    hvg_HSC_v2,
    basis="umap",
    # color=['Mariem_annotation'],
    color=['leiden', 'batch', "scNym_HPC"],
    frameon=False,
    ncols=1,
    palette = "tab20",
    save = "hvg_scVI_HSC_v2_leiden_batch_scNym_HPC.pdf"
)

In [ ]:
hvg_HSC_v2.obsm["X_mde"] = mde(hvg_HSC_v2.obsm["X_scVI"])

In [ ]:
sc.pl.embedding(
    hvg_HSC_v2,
    basis="X_mde",
    color=['leiden', 'batch', "scNym_HPC"],
    frameon=False,
    ncols=1,
)

In [ ]:
sc.set_figure_params(figsize = (8,6))
sc.pl.embedding(
    hvg_HSC_v2,
    basis="umap",
    # color=['Mariem_annotation'],
    color=['leiden', 'batch', "scNym_HPC"],
    frameon=False,
    ncols=1,
    palette = "tab20",
    save = "hvg_scVI_HSC_v2_leiden_manual_annotation_legend.pdf"
)

In [ ]:
# SA_HSC_v2.write_loom(filename = out_dir + "SA_HSC_v2_2508.loom")
hvg_HSC_v2.write_loom(filename = out_dir + "hvg_HSC_v2_2608.loom")

### Modulescoring with HSCs

In [ ]:
HSC_modulescores_quantseq = pd.read_csv(out_dir + "../Rout/SA.hvg.HSC.Quantseq.modulescores.csv", index_col = 0)

In [ ]:
HSC_modulescores_quantseq.columns

In [ ]:
hvg_HSC_v2.obs_names.equals(HSC_modulescores_quantseq.index)

In [ ]:
# hvg_HSC_v2.obs = hvg_HSC_v2.obs.join(HSC_modulescores_quantseq)
sc.set_figure_params(figsize = (8,6))
sc.pl.embedding(
    hvg_HSC_v2,
    basis = "umap", 
    color=[ "leiden", 'Eosinophil', 'Neutrophil', 'Neutrophil_progenitor',
       'Eosinophil_progenitor', 'Granulocyte_Monocyte_progenitor',
       'Common_Myeloid_progenitor'],
    frameon=False,
    ncols=3,
    save = "hvg_scVI_HSC_quantseq_modulescore_plots.pdf"
)

In [ ]:
HSC_modulescores_table2 = pd.read_csv(out_dir + "../Rout/SA.hvg.HSC.table2.modulescores.csv", index_col = 0)

In [ ]:
HSC_modulescores_table2.columns

In [ ]:
# hvg_HSC_v2.obs = hvg_HSC_v2.obs.join(HSC_modulescores_table2)
sc.set_figure_params(figsize = (8,6))
sc.pl.embedding(
    hvg_HSC_v2,
    basis = "umap", 
    color=[ "leiden",'basophil', 'EBM1', 'EBM2', 'eosinophil', 'monocyte',
       'multipotent_progenitor_1', 'multipotent_progenitor_2', 'neutrophil'],
    frameon=False,
    ncols=3,
    save = "hvg_scVI_HSC_table2_modulescore_plots.pdf"
)

In [ ]:
HSC_modulescores_table5 = pd.read_csv(out_dir + "../Rout/SA.hvg.HSC.table5.modulescores.csv", index_col = 0)

In [ ]:
HSC_modulescores_table5.columns

In [ ]:
# hvg_HSC_v2.obs = hvg_HSC_v2.obs.join(HSC_modulescores_table5)
sc.set_figure_params(figsize = (8,6))
sc.pl.embedding(
    hvg_HSC_v2,
    basis = "umap", 
    color=[ "leiden",'eos_stage1', 'eos_stage2', 'eos_stage3', 'eos_stage4', 'eos_stage5'],
    frameon=False,
    ncols=3,
    save = "hvg_scVI_HSC_table5_modulescore_plots.pdf"
)

In [ ]:
HSC_modulescores_panglodb = pd.read_csv(out_dir + "../Rout/SA.hvg.HSC.panglodb.modulescores.csv", index_col = 0)
HSC_modulescores_panglodb.columns

In [ ]:
hvg_HSC_v2.obs = hvg_HSC_v2.obs.join(HSC_modulescores_panglodb)
sc.set_figure_params(figsize = (8,6))
sc.pl.embedding(
    hvg_HSC_v2,
    basis = "umap", 
    color=[ "leiden",'B_cells_memory', 'B_cells', 'Basophils', 'Dendritic_cells',
       'Erythroid-like_and_erythroid_precursor_cells', 'Gamma_delta_T_cells',
       'Macrophages', 'Mast_cells', 'Megakaryocytes', 'Monocytes',
       'Myeloid-derived_suppressor_cells', 'Natural_killer_T_cells',
       'Neutrophils', 'NK_cells', 'Nuocytes', 'Plasma_cells',
       'Plasmacytoid_dendritic_cells', 'Platelets', 'Red_pulp_macrophages',
       'Reticulocytes', 'T_regulatory_cells'],
    frameon=False,
    ncols=3,
    save = "hvg_scVI_HSC_panglodb_modulescore_plots.pdf"
)

### Mature annotations 

In [ ]:
hvg_mature_v2 = hvg_subset[SA_exps_mature.obs_names.intersection(hvg_subset.obs_names)]
# SA_mature_v2 = SA_exps_mature[SA_exps_mature.obs_names.intersection(hvg_subset.obs_names)]

In [ ]:
del hvg_mature_v2.obsm

In [ ]:
hvg_mature_v2.layers

In [ ]:
# sc.pp.normalize_total(hvg_mature_v2, target_sum=1e6)
# sc.pp.log1p(hvg_mature_v2)
sc.tl.pca(hvg_mature_v2, layer="scVI_normalized")
sc.pp.neighbors(hvg_mature_v2, use_rep="X_pca")
sc.tl.umap(hvg_mature_v2, random_state=46468856)
sc.tl.leiden(hvg_mature_v2, n_iterations=-1, resolution=0.5)

In [ ]:
sc.set_figure_params(figsize = (8,6))
sc.pl.embedding(
    hvg_mature_v2,
    basis="umap",
    # color=['Mariem_annotation'],
    color=['leiden', 'batch'],
    frameon=False,
    ncols=1,
    palette = "tab20",
    save = "hvg_scVI_HSC_v2_leiden_manual_annotation_legend.pdf"
)

In [ ]:
out_dir